In [4]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import json
import pandas as pd
import requests
import ast
import re
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [5]:
from fetch_trends import hashtag_trend_info

In [6]:
url = f'https://ads.tiktok.com/business/creativecenter/trends/home/pc/en'
page = requests.get(url)
soup = BeautifulSoup(page.text, features='lxml')

trending_hashtag = []
trending_music = []
trending_creator = []

# get hashtag info
for s in soup.find_all('div', id='hashtagItemContainer'):
    tag = s.find('span', {"class": "titleText--qKHbP"}).text
    stats = s.find_all('span', {"class": "item-value--VAdnq"})
    posts = stats[0].text
    views = stats[1].text
    trending_hashtag.append([tag, posts, views])

# get music info
for s in soup.find_all('div', {"class": "music-name-wrap--FPaLL music-name-wrap--X6m9u"}):
    trending_music.append(s.text)

# get creator info
for s in soup.find_all('div', id='creatorItemContainer'):
    tag = s.find('span', {"class": "music-name--LR+1s music-name--bEzZ1"}).text
    stats = s.find_all('span', {"class": "creator-data-value--CwFQt creator-data-value--S-K2V"})
    posts = stats[0].text
    views = stats[1].text
    trending_creator.append([tag, posts, views])

hashtag = pd.DataFrame(trending_hashtag, columns=['tag', 'posts_count', 'views_count'])
music = pd.DataFrame(trending_music, columns=['music'])
creator = pd.DataFrame(trending_creator, columns=['creator', 'followers_count', 'likes_count'])

find_content = soup.find('script', id="__NEXT_DATA__")
str_content = str(find_content).split('<')[1].split('>')[1]

json_content = json.loads(str_content)['props']['pageProps']

if 'data' in json_content.keys():
    json_content = json_content['data']

hashtag_content = json_content['hashtags']
music_content = json_content['trendMusic']
creator_content = json_content['creatorList']
tiktok_content = json_content['tiktoks']

# gather all data
hashtag = hashtag.merge(pd.DataFrame.from_records(hashtag_content), left_index=True, right_index=True)
music = music.merge(pd.DataFrame.from_records(music_content), left_index=True, right_index=True)
creator = creator.merge(pd.DataFrame.from_records(creator_content), left_index=True, right_index=True)
tiktok = pd.DataFrame.from_records(tiktok_content)

In [7]:
hashtag

,tag,posts_count,views_count,trend,creators,rank,hashtagId,hashtagName,countryInfo,isPromoted,trendingType,publishCnt,videoViews,rankDiff,rankDiffType,industryInfo
0,# springbreak,46K,213M,"[{'time': 1678147200, 'value': 0.5}, {'time': ...","[{'nickName': 'billy', 'avatarUrl': 'https://p...",1,13177,springbreak,"{'id': 'US', 'value': 'United States', 'label'...",False,1,45784,212558426,0.0,2,NaN
1,# balenciaga,61K,13M,"[{'time': 1678147200, 'value': 0.25}, {'time':...","[{'nickName': 'vincenzoboviski', 'avatarUrl': ...",2,198050,balenciaga,"{'id': 'US', 'value': 'United States', 'label'...",False,1,60998,13101288,NaN,4,"{'id': 22000000000, 'value': 'Apparel & Access..."
2,# jordan,24K,75M,"[{'time': 1678147200, 'value': 0.39}, {'time':...","[{'nickName': 'Tee’sWay', 'avatarUrl': 'https:...",3,29510,jordan,"{'id': 'US', 'value': 'United States', 'label'...",False,1,23536,74772086,NaN,4,"{'id': 22000000000, 'value': 'Apparel & Access..."
3,# scream,19K,124M,"[{'time': 1678147200, 'value': 0.43}, {'time':...","[{'nickName': 'omar 🍄', 'avatarUrl': 'https://...",4,14187,scream,"{'id': 'US', 'value': 'United States', 'label'...",False,1,18944,124102827,2.0,3,"{'id': 23000000000, 'value': 'News & Entertain..."
4,# dior,42K,26M,"[{'time': 1678147200, 'value': 0.36}, {'time':...","[{'nickName': 'Jaadiee', 'avatarUrl': 'https:/...",5,112892,dior,"{'id': 'US', 'value': 'United States', 'label'...",False,1,42147,26054270,NaN,4,"{'id': 22000000000, 'value': 'Apparel & Access..."


In [8]:
hashtag['creators_examples'] = hashtag.apply(lambda x: ([e['nickName'] for e in x['creators']]), axis=1)


In [9]:
hashtag

,tag,posts_count,views_count,trend,creators,rank,hashtagId,hashtagName,countryInfo,isPromoted,trendingType,publishCnt,videoViews,rankDiff,rankDiffType,industryInfo,creators_examples
0,# springbreak,46K,213M,"[{'time': 1678147200, 'value': 0.5}, {'time': ...","[{'nickName': 'billy', 'avatarUrl': 'https://p...",1,13177,springbreak,"{'id': 'US', 'value': 'United States', 'label'...",False,1,45784,212558426,0.0,2,NaN,"[billy, Darcy McQueeny, rileycahill]"
1,# balenciaga,61K,13M,"[{'time': 1678147200, 'value': 0.25}, {'time':...","[{'nickName': 'vincenzoboviski', 'avatarUrl': ...",2,198050,balenciaga,"{'id': 'US', 'value': 'United States', 'label'...",False,1,60998,13101288,NaN,4,"{'id': 22000000000, 'value': 'Apparel & Access...","[vincenzoboviski, Gringa Fake, Beth Herren]"
2,# jordan,24K,75M,"[{'time': 1678147200, 'value': 0.39}, {'time':...","[{'nickName': 'Tee’sWay', 'avatarUrl': 'https:...",3,29510,jordan,"{'id': 'US', 'value': 'United States', 'label'...",False,1,23536,74772086,NaN,4,"{'id': 22000000000, 'value': 'Apparel & Access...","[Tee’sWay, SIRROM, samaonsafari]"
3,# scream,19K,124M,"[{'time': 1678147200, 'value': 0.43}, {'time':...","[{'nickName': 'omar 🍄', 'avatarUrl': 'https://...",4,14187,scream,"{'id': 'US', 'value': 'United States', 'label'...",False,1,18944,124102827,2.0,3,"{'id': 23000000000, 'value': 'News & Entertain...","[omar 🍄, screamforghosty | Ghostface, iamsigor]"
4,# dior,42K,26M,"[{'time': 1678147200, 'value': 0.36}, {'time':...","[{'nickName': 'Jaadiee', 'avatarUrl': 'https:/...",5,112892,dior,"{'id': 'US', 'value': 'United States', 'label'...",False,1,42147,26054270,NaN,4,"{'id': 22000000000, 'value': 'Apparel & Access...","[Jaadiee, våñdëń, ElTooLocos]"
